In [1]:
# spacy 패키지 설치
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2023-08-27 14:43:20.276269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 14:43:21.261478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-27 14:43:22.834429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-27 14:43:22.834943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
# torchtext 패키지 설치
!pip install torchtext

# 최신 버전의 Pytorch에서는 Field, BucketIterator 가 torchtext.legacy.data 로 이전되었다고 하는데
# torchtext.legacy.data 로 불러와지지 않아서 옛날 버전으로 변경
# pip => python package 관리 도구 중
!pip install -U torchtext==0.6

# 만약에 런타임 재부팅 하라는 메시지가 나오면, 다시 처음부터 위에서부터 실행

In [3]:
# 라이브러리/모듈 불러오기

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
# Field : 텍스트 데이터를 처리하고 모델에 입력을 공급하기 위한 Field 객체. 텍스트 데이터의 전처리 및 후처리를 정의하는 객체
# BucketIterator : 데이터를 미니배치로 나누어주는 역할. 미니배치 크기, 데이터 정렬 방식 등을 설정할 수 있으며
#                  주어진 'Field' 객체에 따라 텍스트 데이터를 숫자로 변환하여 반환

import spacy
import numpy as np

import random
import math
import time

In [4]:
# 랜덤 SEED 설정
SEED = 1234

# random 모듈 랜덤 SEED 설정
random.seed(SEED)
# numpy 패키지 랜덤 모듈 numpy.random 랜덤 SEED 설정
np.random.seed(SEED)
# 파이토치 CPU 기반 랜덤 모듈 SEED 설정
torch.manual_seed(SEED)
# 파이토치 CUDA 랜덤 모듈 SEED 설정
torch.cuda.manual_seed(SEED)
# NVIDA cuDNN을 사용하는 경우 SEED 설정
torch.backends.cudnn.deterministic = True

In [5]:
# 공식 웹 사이트: https://spacy.io/models/de

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화(tokenization)

In [8]:
# 간단한 토큰화(tokenization) 기능
# 문장에 존재하는 각 word 레벨을 토큰으로, 고유한 인덱스를 부여
# 토큰(token) 에는 중요한 의미를 담는 어떤 데이터도 해당될 수 있음

tokenized = spacy_en.tokenizer("Song computer engineering tistory blog!  Song electric engineering naver blog.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

# 토큰화를 통해 문장을 단어의 나열(sequence of word)가 아닌 토큰의 나열(sequence of token)으로 해석
# token과 word 의 다른 점 : Song, computer, engineering, blog 등은 word 이면서 token 이지만, !, ., , 는 word는 아니고 token 이다.

인덱스 0: Song
인덱스 1: computer
인덱스 2: engineering
인덱스 3: tistory
인덱스 4: blog
인덱스 5: !
인덱스 6:  
인덱스 7: Song
인덱스 8: electric
인덱스 9: engineering
인덱스 10: naver
인덱스 11: blog
인덱스 12: .


In [12]:
# seq2seq 논문에서는 입력 문장을 넣을 때 토큰 순서를 바꿔서 넣기 때문에 문장을 입력으로 받아 토큰화 후 뒤집어 반환하는 토큰화 함수 정의
# 독일어 → 영어 번역을 진행하기 때문에 독일어만 뒤집는다. (인코더에 들어가는 입력 데이터)

# 독일어(Deutsch) 문장을 토큰화한 뒤에 순서를 뒤집는 함수
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

tokenized = tokenize_en("Seq2Seq paper Code Pratice with Song.")
print(tokenized) # 문장이 들어오면, 단어 단위, 특수문자 단위로 쪼개준다고 생각하면 편함.

['Seq2Seq', 'paper', 'Code', 'Pratice', 'with', 'Song', '.']


In [15]:
# 토큰화 인수를 각각에 대한 토큰화 함수로 정의
# init_token: <sos> 토큰
# eos_token: <eos> 토큰
# lower : 모두 소문자로 처리 → 전체 토큰 수를 줄일 수 있기 때문에 모두 소문자로 처리
# 독일어는 Tource(SRC) Field
# 영어는 Target(TRG) Field

SRC = Field(tokenize = tokenize_de,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [18]:
import torchtext

# 현재 다운로드 링크가 동작하지 않으므로, 동작되는 URL로 변경, 변경해도 text_dataset은 다운되지 않아 train, valid 만 다운로드
torchtext.datasets.Multi30k.urls = [
    r"https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz",
    r"https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz",
    r"https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz",
]

print(torchtext.datasets.Multi30k.urls)

['https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz', 'https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz', 'https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz']


In [21]:
from torchtext.datasets import Multi30k

# 총 30,000개의 (독일어, 영어) 쌍으로 구성된 데이터 세트
train_dataset, valid_dataset = Multi30k.splits(exts=(".de", ".en"),
                                               fields=(SRC, TRG),
                                               root='.custom_data',
                                               train="train",
                                               validation="val",
                                               test=None)

In [22]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개


In [25]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src']) # 순서를 바꿨기 때문에, 온점이 제일 첫째로 나온다.
print(vars(train_dataset.examples[30])['trg']) # 정상적인 순서

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [29]:
# 최소 두 번 이상 등장한 단어만 기록
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

# 한 번만 등장하는 희소한 단어는? unk 토큰으로 처리된다.
# 자연어 처리 분야에서 "특수 토큰"은 보통 4가지가 존재
  # 1) sos : 문장의 시작을 알림
  # 2) eos : 문장의 끝을 알림
  # 3) unk : 처음 보는 단어를 알림
  # 4) pad : 짧은 문장의 경우 뒤쪽을 패딩 토큰으로 채움

len(SRC): 7853
len(TRG): 5893


In [34]:
# string to index (stoi)
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos> 토큰 : 2
print(TRG.vocab.stoi["<eos>"]) # <eos> 토큰 : 3
print(TRG.vocab.stoi["hello"]) # 있는 단어는 해당 인덱스가 나타난다.
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


In [35]:
# torch.device 정의
# torchText 에게 텐서를 GPU에 넣었는지의 여부를 알려주는 데 사용
# GPU가 감지되면 True를 반환하는 torch.cuda.is_available() 함수 사용

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Iterator 정의
# src attribute(숫자화된 소스 문장의 텐서)와 trg attribute(숫자화된 타깃 문장의 텐서)를 가지는 data batch를 반환
# 문장들에서 패딩의 양을 최소화하는 방식으로 배치를 만들기 위해 표준 Iterator 대신 BucketIterator 사용

BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset),
    batch_size=BATCH_SIZE,
    device=device)

In [37]:
# Iterator 확인해보기
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"인덱스 {i}: {src[i][0].item()}")

    # 첫 번째 배치만 확인
    break


# 첫 번째 배치 크기: torch.Size([27, 128])
# 현재 배치에 포함된 문장의 수가 128개 (batch_size = 128)
# 27 이 의미하는 것은 128개의 문장 중 가장 긴 문장의 토큰 개수를 의미

첫 번째 배치 크기: torch.Size([33, 128])
인덱스 0: 2
인덱스 1: 4
인덱스 2: 93
인덱스 3: 1784
인덱스 4: 20
인덱스 5: 998
인덱스 6: 17
인덱스 7: 101
인덱스 8: 17
인덱스 9: 35
인덱스 10: 9
인덱스 11: 214
인덱스 12: 33
인덱스 13: 12
인덱스 14: 217
인덱스 15: 67
인덱스 16: 8
인덱스 17: 3
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1
인덱스 29: 1
인덱스 30: 1
인덱스 31: 1
인덱스 32: 1


### **Building the Seq2Seq Model**
---

In [39]:
# 인코더(Encoder) 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 context vector 를 반환
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        # 각 토큰별로 |V| 크기의 one-hot 벡터
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보. 각 계층의 최종 hidden state
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보. 각 계층의 최종 cell state

        # hidden, cell 은 가장 마지막 토큰을 넣었을 때 나온 hidden state가 되고, 이것이 디코더의 초기 입력이 된다.

        # context vector 반환
        return hidden, cell

seq2seq2.png

In [40]:
# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding) 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # FC 레이어 (인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
    def forward(self, input, hidden, cell):
        # seq2seq는 항상 (1) 이전에 출력한 단어와 (2) 이전까지의 정보를 담은 hidden state가 입력으로 들어감
        # 그러므로, 이전에 출력했던 모든 단어에 대한 것은 필요하지 않다.
        # 디코더를 eos 가 나올 때까지 여러 번 forward해서 전체 문자열을 출력.

        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [레이어 개수, 배치 크기, 히든 차원]
        # cell = context: [레이어 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)

        # PyTorch에서 squeeze()는 차원 축소 (불필요한 차원 제거)
        # PyTorch에서 unsqueeze()는 차원 늘리기 (크기가 1인 axis 추가)

        # input: [단어 개수 = 1, 배치 크기]

        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보. 각 계층의 최종 hidden state
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보. 각 계층의 최종 cell state

        # 단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [배치 크기, 출력 차원]

        # (현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
        return prediction, hidden, cell

seq2seq3.png

In [41]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio 를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 문맥 벡터(context vector)를 추출
        hidden, cell = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            # eos 가 나올 때까지 재귀적으로 사용
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # 현재 구조상, 만약에 디코더 입장에서 이전 단어가 틀리면, 앞으로의 단어도 계속 틀릴 가능성이 매우 높음
            #  => 이 상태로 그대로 학습하면, 너무 많이 틀려서 loss가 과도하게 커짐
            # "그래서, 강제로 특정 확률에 해당할 때는 "앞으로의 단어"를 고정해서 정답으로 알려주는 방법을 사용" (teacher forcing 방법)

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio # random.random()은 [0, 1] uniform하게 값을 뽑음 => 평균 0.5

            # teacher_force => recursive 도중에 정답 자체를 교정
            # 만약 teacher_force의 값이 True라면, 강제로 "정답"을 다음 입력으로 넣어준다. (강제로 정답을 맞추도록 forcing)
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

seq2seq4.png


### **Training the Seq2Seq Model**
---


In [42]:
# 모델 초기화

# 입력과 출력 차원은 어휘의 전체 크기가 결정
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256 # 각 토큰(단어)가 표현되는 차원
DECODER_EMBED_DIM = 256 # 각 토큰(단어)가 표현되는 차원
HIDDEN_DIM = 512 # LSTM 내부에서 사용되는 hidden vector의 차원
N_LAYERS = 2 # LSTM을 얼마나 높게 쌓을 것인지
ENC_DROPOUT_RATIO = 0.5 # dropout => 모델 성능 향상 기법 중 하나
DEC_DROPOUT_RATIO = 0.5 # dropout => 모델 성능 향상 기법 중 하나

In [43]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

In [44]:
# 모델 가중치 초기화
# 논문에서 -0.08 ~ 0.08 사이의 균일한 분포에서 모든 가중치를 초기화한다고 함.
# 논문의 내용대로 모델에 적용시키는 init_weights 함수를 정의한다.
# 함수는 모델 내의 모든/하위 모듈에서 호출되고, 각 모듈에 대해 모든 매개변수를 순환시키고 균일 분포에서 표본을 추출 (with nn.init.uniform_)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [45]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [46]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]

        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [47]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드 -> dropout이 꺼진다
    epoch_loss = 0

    # torch.no_grad() => 이 안에 들어가는 모델 및 입력에 대해서는 "기울기 추적"을 하지 않음. 메모리 사용량이 줄고 속도가 빨라진다.
    # (inference = 학습된 모델을 추론할 때 / 쓸 때만 사용)
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]

            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [48]:
# epoch 당 소요시간 측정 함수
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [49]:
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 43s
	Train Loss: 5.050 | Train PPL: 156.041
	Validation Loss: 4.935 | Validation PPL: 139.008
Epoch: 02 | Time: 0m 41s
	Train Loss: 4.502 | Train PPL: 90.167
	Validation Loss: 4.826 | Validation PPL: 124.744
Epoch: 03 | Time: 0m 42s
	Train Loss: 4.213 | Train PPL: 67.552
	Validation Loss: 4.690 | Validation PPL: 108.842
Epoch: 04 | Time: 0m 40s
	Train Loss: 4.005 | Train PPL: 54.860
	Validation Loss: 4.615 | Validation PPL: 100.942
Epoch: 05 | Time: 0m 41s
	Train Loss: 3.838 | Train PPL: 46.435
	Validation Loss: 4.345 | Validation PPL: 77.097
Epoch: 06 | Time: 0m 41s
	Train Loss: 3.682 | Train PPL: 39.738
	Validation Loss: 4.281 | Validation PPL: 72.295
Epoch: 07 | Time: 0m 41s
	Train Loss: 3.549 | Train PPL: 34.789
	Validation Loss: 4.105 | Validation PPL: 60.631
Epoch: 08 | Time: 0m 42s
	Train Loss: 3.379 | Train PPL: 29.329
	Validation Loss: 4.121 | Validation PPL: 61.600
Epoch: 09 | Time: 0m 45s
	Train Loss: 3.271 | Train PPL: 26.348
	Validation Loss: 4.006 | V

In [51]:
model.load_state_dict(torch.load('seq2seq.pt'))

# text 데이터를 정상으로 불러왔다면 가장 좋은 validation loss 를 기록한 모델(state_dict)을 불러와 성능 측정할 수 있음
# test_loss = evaluate(model, test_iterator, criterion)
# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

# test 데이터가 없는 상황이므로, 제일 잘 학습된 validation model 을 확인하고 마무리
valid_loss = evaluate(model, valid_iterator, criterion)
print(f'| Validation Loss: {valid_loss:.3f} | Test PPL: {math.exp(valid_loss):7.3f} |')

| Validation Loss: 3.979 | Test PPL:  53.454 |
